# Router
This notebook demonstrates how to use the RouterChain paradigm to create a chain that dynamically selects the next chain to use for a given input.

Router chains are made up of two components:
- The RouterChain itself (responsible for selecting the next chain to call)
- destination_chains: chains that the router chain can route to